# Keras Functional API for Multi-Input Neural Networks
This notebook will demonstrate how to load numerical, categorical, and image data, preprocessit, and use it for a multi-input model for Keras.

Examples of Data Types:
    * Numeric/Continuous Values: age, heart rate, blood pressure
    * Categorical Values: gender, ethnicity
    * Image Data: MRI, X-Ray

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelBinarizer, MinMaxScaler
from sklearn.model_selection import train_test_split

import glob
import cv2
import os
import argparse
import locale

from keras.models import Sequential, Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Dropout, Dense
from keras.optimizers import Adam
from keras.layers import concatenate
from keras.layers import Flatten, Input

Using TensorFlow backend.


In [2]:
# Set the path to the directory of the dataset
Data_Directory = '/Users/edenmolina/Documents/AI-Playground/Houses Dataset'

In [3]:
# This function will load the data and return only zipcodes with <25 houses
def load_house_attributes(inputPath, cols):
    """
    inputPath: string path to the text file containing the house data
    cols: array of column names
    """
    # Set column names and load the text data
    cols = cols
    df = pd.read_csv(inputPath, sep=" ", header=None, names=cols)

    # determine (1) the unique zip codes and (2) the number of data
    # points with each zip code
    zipcodes = df["zipcode"].value_counts().keys().tolist()
    counts = df["zipcode"].value_counts().tolist()

    # loop over each of the unique zip codes and their corresponding
    # count
    for (zipcode, count) in zip(zipcodes, counts):
        # the zip code counts for our housing dataset is *extremely*
        # unbalanced (some only having 1 or 2 houses per zip code)
        # so let's sanitize our data by removing any houses with less
        # than 25 houses per zip code
        if count < 25:
            idxs = df[df["zipcode"] == zipcode].index
            df.drop(idxs, inplace=True)

    # return dataframe with houses with less than 25 houses per zipcode
    return df

In [51]:
# This function will process the normalize the data and one hot encode the zipcodes and create training and testing data
def process_house_attributes(df, train, test, continuous_columns):
    # Initiates the column names of the continuous data
    continuous = continuous_columns

    # Perform min-max scaling so each feature value is [0, 1]
    cs = MinMaxScaler()
    trainContinuous = cs.fit_transform(train[continuous])
    testContinuous = cs.transform(test[continuous])
    print ('Data is now scaled between 0 and 1')
    
    # One-hot encode the zip code categorical data
    zipBinarizer = LabelBinarizer().fit(df['zipcode'])
    trainCategorical = zipBinarizer.transform(train['zipcode'])
    testCategorical = zipBinarizer.transform(test['zipcode'])
    print ('Data is now one-hot encoded')
    
    # Stack the data together
    trainX = np.hstack([trainCategorical, trainContinuous])
    testX = np.hstack([testCategorical, testContinuous])
    print ('Continuous and Categorical Data now stacked')
    print (np.shape(trainX), np.shape(testX))
    
    # Export the scaled and encoded data
    return (trainX, testX)

In [5]:
# This function will load the images, generate a montage image from the four images
# images will be arranged from bathroom, bedroom, kichen, frontal
# append all home montage from four photos
def load_house_images(df, inputPath):
    """
    df: pandas dataframe of house data
    inputPath: path to the dataset
    """
    # initialize our images
    images = []
    
    # Loop over the indecies of the houses
    for i in df.index.values:
        # find four images for the house and sort by file path
        # ensure that the four images are ALWAYS in the same order
        basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])

        housePaths = sorted(list(glob.glob(basePath)))

        # Initialize list of input images along with the output image
        # Images are in color, there fore there are 3 channels
        inputImages = []
        outputImage = np.zeros((64, 64, 3), dtype = 'uint8')

        # Loop over the input house paths
        for housePath in housePaths:
            image = cv2.imread(housePath)

            # resize image
            image = cv2.resize(image, (32, 32))
            inputImages.append(image)

            # tile the four images in the output image
            # first image goes in top-right
            # second image goes in top left
            # third image goes in bottom right
            # fourth goes in bottom left

        outputImage[0:32, 0:32] = inputImages[0]
        outputImage[0:32, 32:64] = inputImages[1]
        outputImage[32:64, 32:64] = inputImages[2]
        outputImage[32:64, 0:32] = inputImages[3]

        # add tiled image to the set of images the network will be trained on
        images.append(outputImage)

    return np.array(images)

# Construct Multi-Layer Perceptron (MLP) and Convolutional Neural Network (Covnet)

In [6]:
def create_mlp(dim, regress = False):
    """
    Create layers of Multi-Layer Perceptron
    can return tensor dim 8 or can regress to output scalar
    """
    # Define MLP as 2 dense layers
    model = Sequential()
    model.add(Dense(8, input_dim = dim, activation = 'relu'))
    model.add(Dense(4, activation = 'relu'))
    
    # Check if regression node should be added
    if regress:
        model.add(Dense(1, activation='linear'))
        
    # Return the Model
    return model

In [7]:
def create_cnn(width, height, depth, filters = (16, 32, 64), regress = False):
    """
    Handles the image data
    width: int width of the input image in pixels
    height: int height of the input image in pixels
    depth: int number of channels in image (for RGB it is 3)
    filters: tuple of progressively larger filters to the network can learn to discriminate features
    regress: bool inidicating whether or not a fully connected linear activation layer will be appended to the CNN for regression
    """
    # Initialize input shape and channel dimension
    # Assumes a 'channels last' ordering for the TensorFlow backend
    inputShape = (height, width, depth)
    chanDim = -1
    
    # Model Input
    inputs = Input(shape = inputShape)
    
    # Loop over the filters
    for (i, f) in enumerate(filters):
        # If this is the first convolutional layer then set the input accordingly
        if i == 0:
            x = inputs
        
        # Conv > Relu > BN > Pool
        x = Conv2D(f, (3, 3), padding = 'same')(x)
        x = Activation('relu')(x)
        x = BatchNormalization(axis = chanDim)(x)
        x = MaxPooling2D(pool_size = (2, 2))(x)
        
        # Flatten the volume
        # FC > Relu > BN > Dropout
        x = Flatten()(x)
        x = Dense(16)(x)
        x = Activation('relu')(x)
        x = BatchNormalization(axis = chanDim)(x)
        x = Dropout(.5)(x)
        
        # Apply another FC layer to match the nodes outputted by the MLP
        x = Dense(4)(x)
        x = Activation('relu')(x)
        
        # check to see if the regression node should be added
        if regress:
            x = Dense(1, activation="linear")(x)
            
        # construct the CNN
        model = Model(inputs, x)
        
        # Return CNN
        return model

--------

In [52]:
# Firectory of the text file and columns of the data
inputPath = '%s/HousesInfo.txt'%Data_Directory
cols = ['bedrooms', 'bathrooms', 'area', 'zipcode', 'price']
continuous_cols = ['bedrooms', 'bathrooms', 'area']

In [53]:
df = load_house_attributes(inputPath, cols)
df.head()

,bedrooms,bathrooms,area,zipcode,price
30,5,3.0,2520,93446,789000.0
32,3,2.0,1802,93446,365000.0
39,3,3.0,2146,93446,455000.0
80,4,2.5,2464,91901,599000.0
81,2,2.0,1845,91901,529800.0


In [54]:
images = load_house_images(df, Data_Directory)

In [55]:
# Normalize the images
images = images / 255.0

In [56]:
print("[INFO] processing data...")
# Split into train and test data
(trainAttrX, testAttrX, trainImagesX, testImagesX) = train_test_split(df, images, test_size=0.25, random_state=42)
print ("Done!")

[INFO] processing data...
Done!


In [57]:
# find the largest house price in the training set and use it to
# scale our house prices to the range [0, 1] (will lead to better
# training and convergence)
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice

In [58]:
(trainAttrX, testAttrX) = process_house_attributes(df, trainAttrX, testAttrX, continuous_cols)

Data is now scaled between 0 and 1
Data is now one-hot encoded
Continuous and Categorical Data now stacked
(271, 10) (91, 10)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# Creating the network

In [70]:
# create the MLP and CNN models
features_mlp = trainAttrX.shape[1]

mlp = create_mlp(features_mlp, regress=False)
cnn = create_cnn(64, 64, 3, regress=False)
mlp.output, cnn.output

(<tf.Tensor 'dense_20/Relu:0' shape=(?, 4) dtype=float32>,
 <tf.Tensor 'activation_12/Relu:0' shape=(?, 4) dtype=float32>)

In [64]:
# create the input to our final set of layers as the *output* of both
# the MLP and CNN
combinedInput = concatenate([mlp.output, cnn.output])

In [65]:
# our final FC layer head will have two dense layers, the final one
# being our regression head
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

In [66]:
# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the
# predicted price of the house)
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

In [73]:
np.shape(trainAttrX),np.shape(trainImagesX), np.shape(trainY)

((271, 10), (271, 64, 64, 3), (271,))

In [67]:
# compile the model using mean absolute percentage error as our loss,
# implying that we seek to minimize the absolute percentage difference
# between our price *predictions* and the *actual prices*
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
 
# train the model
print("[INFO] training model...")
model.fit([trainAttrX, trainImagesX], trainY,
          validation_data=([testAttrX, testImagesX], testY),
          epochs=200, batch_size=8)

# make predictions on the testing data
print("[INFO] predicting house prices...")
preds = model.predict([testAttrX, testImagesX])

[INFO] training model...
Train on 271 samples, validate on 91 samples
Epoch 1/200
271/271 [==============================] - 2s 7ms/step - loss: 722.6511 - val_loss: 1417.5199
Epoch 2/200
271/271 [==============================] - 1s 3ms/step - loss: 357.0722 - val_loss: 920.2435
Epoch 3/200
271/271 [==============================] - 1s 2ms/step - loss: 241.5115 - val_loss: 425.7796
Epoch 4/200
271/271 [==============================] - 1s 2ms/step - loss: 155.4837 - val_loss: 89.6075
Epoch 5/200
271/271 [==============================] - 1s 2ms/step - loss: 122.5273 - val_loss: 62.5164
Epoch 6/200
271/271 [==============================] - 1s 2ms/step - loss: 73.3599 - val_loss: 56.2964
Epoch 7/200
271/271 [==============================] - 1s 3ms/step - loss: 100.4226 - val_loss: 53.8034
Epoch 8/200
271/271 [==============================] - 1s 3ms/step - loss: 67.9812 - val_loss: 53.6136
Epoch 9/200
271/271 [==============================] - 1s 3ms/step - loss: 64.1387 - val_loss: 5

Epoch 79/200
271/271 [==============================] - 1s 3ms/step - loss: 24.2815 - val_loss: 29.6796
Epoch 80/200
271/271 [==============================] - 1s 3ms/step - loss: 23.9174 - val_loss: 29.5988
Epoch 81/200
271/271 [==============================] - 1s 3ms/step - loss: 23.5228 - val_loss: 30.7539
Epoch 82/200
271/271 [==============================] - 1s 3ms/step - loss: 28.0074 - val_loss: 116536.2888
Epoch 83/200
271/271 [==============================] - 1s 3ms/step - loss: 26.0843 - val_loss: 16625.7750
Epoch 84/200
271/271 [==============================] - 1s 3ms/step - loss: 26.2137 - val_loss: 91.2944
Epoch 85/200
271/271 [==============================] - 1s 3ms/step - loss: 25.3599 - val_loss: 29.5449
Epoch 86/200
271/271 [==============================] - 1s 3ms/step - loss: 23.4791 - val_loss: 28.6680
Epoch 87/200
271/271 [==============================] - 1s 3ms/step - loss: 25.8964 - val_loss: 28.3027
Epoch 88/200
271/271 [==============================] - 1

271/271 [==============================] - 1s 3ms/step - loss: 16.5334 - val_loss: 22.1678
Epoch 158/200
271/271 [==============================] - 1s 3ms/step - loss: 19.8336 - val_loss: 22.7745
Epoch 159/200
271/271 [==============================] - 1s 3ms/step - loss: 17.9555 - val_loss: 70.3851
Epoch 160/200
271/271 [==============================] - 1s 3ms/step - loss: 17.3697 - val_loss: 2494.2922
Epoch 161/200
271/271 [==============================] - 1s 3ms/step - loss: 18.3983 - val_loss: 64.5188
Epoch 162/200
271/271 [==============================] - 1s 3ms/step - loss: 18.0819 - val_loss: 46.3388
Epoch 163/200
271/271 [==============================] - 1s 3ms/step - loss: 15.0021 - val_loss: 23.4511
Epoch 164/200
271/271 [==============================] - 1s 3ms/step - loss: 16.8254 - val_loss: 23.1208
Epoch 165/200
271/271 [==============================] - 1s 3ms/step - loss: 16.4590 - val_loss: 25.3521
Epoch 166/200
271/271 [==============================] - 1s 3ms/ste

In [68]:
# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)
 
# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)
 
# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
    locale.currency(df["price"].mean(), grouping=True),
    locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] avg. house price: $533,388.27, std house price: $493,403.08
[INFO] mean: 21.25%, std: 27.70%
